In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [3]:
data_dir = r'C:\Users\suyog\Downloads\custom_data'
image_size = (224, 224)  # Adjust to your preferred image size
batch_size = 32
num_classes = 2  # Specify the number of classes in your dataset

In [4]:
data_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2,
    validation_split=0.2
)

In [5]:
train_generator = data_gen.flow_from_directory(
    data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

Found 320 images belonging to 1 classes.


In [6]:
validation_generator = data_gen.flow_from_directory(
    data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

# Build and compile the model (MobileNetV2 with fine-tuning)
base_model = keras.applications.MobileNetV2(input_shape=image_size + (3,), include_top=False, weights='imagenet')
base_model.trainable = False  # Freeze the base model

Found 80 images belonging to 1 classes.
9406464/9406464 [==============================] - 1s 0us/step


In [7]:
model = keras.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [8]:
# Train the model
epochs = 10  # Adjust the number of epochs as needed
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=epochs,
    verbose=2
)

Epoch 1/10
10/10 - 32s - loss: 1.8488 - accuracy: 0.5250 - val_loss: 1.4934 - val_accuracy: 0.5250 - 32s/epoch - 3s/step
Epoch 2/10
10/10 - 16s - loss: 1.7922 - accuracy: 0.5437 - val_loss: 1.5054 - val_accuracy: 0.5500 - 16s/epoch - 2s/step
Epoch 3/10
10/10 - 32s - loss: 2.0301 - accuracy: 0.4406 - val_loss: 1.5183 - val_accuracy: 0.5000 - 32s/epoch - 3s/step
Epoch 4/10
10/10 - 22s - loss: 2.2431 - accuracy: 0.4531 - val_loss: 1.5095 - val_accuracy: 0.4875 - 22s/epoch - 2s/step
Epoch 5/10
10/10 - 24s - loss: 2.6391 - accuracy: 0.4750 - val_loss: 1.5493 - val_accuracy: 0.6125 - 24s/epoch - 2s/step
Epoch 6/10
10/10 - 13s - loss: 2.9521 - accuracy: 0.4563 - val_loss: 1.5210 - val_accuracy: 0.4750 - 13s/epoch - 1s/step
Epoch 7/10
10/10 - 13s - loss: 3.2580 - accuracy: 0.4875 - val_loss: 1.5246 - val_accuracy: 0.3375 - 13s/epoch - 1s/step
Epoch 8/10
10/10 - 13s - loss: 3.6753 - accuracy: 0.5031 - val_loss: 1.6448 - val_accuracy: 0.2000 - 13s/epoch - 1s/step
Epoch 9/10
10/10 - 13s - loss: 4

In [9]:
model.save('custom_object_classifier.h5')

In [14]:
import cv2
import tensorflow as tf

# Load the saved model
model = tf.keras.models.load_model("custom_object_classifier.h5")

In [27]:
import cv2
import tensorflow as tf
import numpy as np

# Define class names
class_names = ["knife", "guns", "others"]

# Load the custom object classification model
model = tf.keras.models.load_model("custom_object_classifier.h5")

# Define the preprocessing function
def preprocess_frame(frame, input_size=(224, 224)):
    # Resize the frame to the specified input size
    frame = cv2.resize(frame, input_size)

    # Normalize pixel values to be in the range [0, 1]
    frame = frame / 255.0

    # Expand dimensions to create a batch of size 1
    frame = np.expand_dims(frame, axis=0)

    return frame

# Open the video stream
cap = cv2.VideoCapture(0)  # To open the default camera (0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Preprocess the frame
    processed_frame = preprocess_frame(frame)

    # Perform object classification
    predictions = model.predict(processed_frame)
    class_id = np.argmax(predictions[0])
    class_name = class_names[class_id]

    # Display the classification result on the frame
    cv2.putText(frame, f"Class: {class_name}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Display the processed frame
    cv2.imshow("Object Classification", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 70ms/step


In [29]:
import cv2
import numpy as np

# Load YOLOv3 model and classes
net = cv2.dnn.readNet(r"C:\Users\suyog\violence_detection_app\yolov3.weights", r"C:\Users\suyog\violence_detection_app\yolov3.cfg")

with open(r"C:\Users\suyog\violence_detection_app\coco.names") as f:
    classes = [line.strip() for line in f.readlines()]

In [30]:

# Open the video stream
cap = cv2.VideoCapture(0)  # To open the default camera (0)
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Get the height and width of the frame
    height, width, _ = frame.shape

    # Preprocess the frame for YOLOv3
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(net.getUnconnectedOutLayersNames())

    # Lists to store detected objects' information
    class_ids = []
    confidences = []
    boxes = []

    # Process the detected objects
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]

            if confidence > 0.5:  # Adjust the confidence threshold as needed
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # Apply non-maximum suppression to filter out overlapping objects
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    # Draw bounding boxes and labels on the frame
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            confidence = confidences[i]
            color = (0, 255, 0)  # BGR color for the bounding box (green in this case)
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv2.putText(frame, f"{label} {confidence:.2f}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Display the frame with object detection
    cv2.imshow("YOLOv3 Object Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
